# VacationPy
----

In [2]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json

# Import API key
from api_keys import google_maps_key

### Import Part I results and store into DataFrame

In [3]:
df = pd.read_csv("../WeatherPy/Cities Weather Log.csv")
df.head()

,Unnamed: 0,City,Country,Latitude,Longitude,Current Temperature,Humidity,Cloudiness,Wind Speed,Date
0,0,Ozona,US,30.40,-101.04,73.3,64,1,4.10,2020-10-26 00:53:12
1,1,Seyfūllīn,KZ,53.44,68.94,45.3,87,100,5.97,2020-10-26 00:53:12
2,2,Sesheke,ZM,-17.13,24.75,68.8,34,0,1.49,2020-10-26 00:53:13
3,3,Mweka,CD,-4.52,21.44,69.4,99,100,0.67,2020-10-26 00:53:13
4,4,Bowen,AR,-34.47,-67.10,64.7,28,0,2.61,2020-10-26 00:53:13


### Create Humidity Heatmap
* Gmaps (a Python library client for Google Maps services) is used to plot temperature data as a heat map.

In [4]:
# Configure gmaps with Google Maps API key.
gmaps.configure(api_key=google_maps_key)

In [5]:
# Parse Latitude, Longitude, and Current Temperature colums into two new DataFrames for plotting.
# A gmaps heat map cannot display negative values, so temperatures are shifted 25 degrees to ensure all
# temperatures are positive.
locations = df[["Latitude", "Longitude"]].astype(float)
temperatures = df["Current Temperature"].astype(float)
temperatures_shifted = df["Current Temperature"] + 25

In [6]:
# Create a heat map layer with Current Temperature as heat map weights.
figure = gmaps.figure(center = (0, 0), zoom_level = 2)
# Set map layer symbology and translucency.
layer = gmaps.heatmap_layer(locations, weights = temperatures_shifted, dissipating = False, max_intensity = 175, point_radius = 5, opacity = 0.25)
figure.add_layer(layer)
figure

Figure(layout=FigureLayout(height='420px'))

### Select ideal vacation destinations based on weather criteria

In [7]:
ideal_df = df.loc[(df["Current Temperature"] > 0) & (df["Cloudiness"] > 5) & (df["Cloudiness"] < 100) & (df["Humidity"] < 50)]
ideal_df

,Unnamed: 0,City,Country,Latitude,Longitude,Current Temperature,Humidity,Cloudiness,Wind Speed,Date
40,40,Dunnigan,US,38.93,-121.93,65.9,15,20,10.80,2020-10-26 00:53:34
77,77,Whyalla,AU,-32.89,137.71,57.2,44,90,5.10,2020-10-26 00:53:52
92,92,Daintree,AU,-16.24,145.13,86.7,38,57,2.57,2020-10-26 00:54:00
102,102,Maradi,NE,14.67,7.42,79.3,17,74,4.00,2020-10-26 00:54:07
114,114,Ardmore,CA,54.39,-110.46,24.8,45,75,4.10,2020-10-26 00:54:15
119,119,Khomas,NaN,-23.01,17.89,67.2,41,14,1.58,2020-10-26 00:54:17
126,126,Aligoundiri,NE,13.38,12.43,80.8,19,73,4.30,2020-10-26 00:54:20
168,168,Condamine,AU,-26.95,150.12,71.6,49,78,3.60,2020-10-26 00:54:50
226,226,Naghadeh,IR,37.18,45.38,49.1,37,75,3.10,2020-10-26 00:55:31
277,277,San,ML,13.21,-5.37,79.8,17,54,2.95,2020-10-26 00:56:01


### Use Google Maps Places API to find parks and gardens near each destination city

In [8]:
ideal_df["Garden Name"] = ""

<ipython-input-8-1b90796047fd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_df["Garden Name"] = ""


In [9]:
# Test an API call.
search_params = {
          "location": "33.4, -112", "radius": 100000, 
          "type": "park", "keyword": "garden", 
          "key": google_maps_key
          }
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
response = requests.get(base_url, params=search_params).json()
print(json.dumps(response, indent=4, sort_keys=True))

{
    "html_attributions": [],
    "next_page_token": "AcnaJv3ZXpYes5a4F4ukebuREsWtsFeJ8DITlCzuJGrZ3reAawY4diedYK1gmFgT85SuRsC6R0SzVDmP9aINtrFYolrmGgBc9GDDfiuopmBu7tpFkb_GJU5W6HWcolM62BOM6yUBQdpYLPuRQw-245R1aVcKObgWAF9j0X8_Q9s0MC34CMRHuKhZAbLM3T1USq1FARsXXSxXS5TQ50ZrMlnUBREd9oNVX95Ss9-BlxvT7ysApZsJdWkOZXqgADImZfG5u0tafILdhkcEmXgZFpzUAyy59udFA99JCGfAoR0VxRYVE0cc_-1ZHsYHYVOgRp0yR9HP40BOQCwq8oJT5r-vsMAJzBYVMuzZjlugrFV7IfreDUGxpxKSry05o9yrwKBY18XVmz3V8Psz72CZ3swxomW-NsPl4vNpmjDZUew3-O776ENGzKmAjP_6KA",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 33.47698099999999,
                    "lng": -112.3142904
                },
                "viewport": {
                    "northeast": {
                        "lat": 33.47846282989271,
                        "lng": -112.3129409701073
                    },
                    "southwest": {
                        "lat": 33.47576

In [14]:
# Store base URL for API call.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Loop through ideal_df to gather parameters into a dictionary, make api calls to find gardens near each destination,
# and store garden names to ideal_df.
for index, row in ideal_df.iterrows():
    
    # Create search parameter dictionary to pass to API call
    search_params = {
                     "location": str(row["Latitude"]) + ", " + str(row["Longitude"]), "radius": 10000000, 
                     "type": "park", "keyword": "garden", 
                     "key": google_maps_key
                    }
    
    # Make API call
    response = requests.get(base_url, params=search_params).json()
    
    # Print the response or a message if no garden is found
    try:
        ideal_df.loc[index, "Garden Name"] = response["results"][2]["name"]
    except:
        ideal_df.loc[index, "Garden Name"] = np.nan

In [34]:
gardens_df = ideal_df.dropna(axis = 0, how = 'any')
gardens_df

C:\Users\vsnix\anaconda3\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,City,Country,Latitude,Longitude,Current Temperature,Humidity,Cloudiness,Wind Speed,Date,Garden Name
0,Maradi,NE,14.67,7.42,79.3,17,74,4.00,2020-10-26 00:54:07,Roman Park
1,Aligoundiri,NE,13.38,12.43,80.8,19,73,4.30,2020-10-26 00:54:20,Roman Park
2,San,ML,13.21,-5.37,79.8,17,54,2.95,2020-10-26 00:56:01,Roman Park
3,Hadjer-Lamis Region,TD,12.99,16.29,82.4,16,87,5.55,2020-10-26 00:56:46,Administration park


### Plot gardens on Current Temperature heat map

In [39]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Garden Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store information from gardens_df for info box
garden_info = [info_box_template.format(**row) for index, row in gardens_df.iterrows()]

# Store locations to plot on map
locations = gardens_df[["Latitude", "Longitude"]]

# Create marker layer
marker_layer = gmaps.marker_layer(locations, info_box_content=garden_info, display_info_box=True)

In [42]:
# Add marker layer to heat map and display
figure.add_layer(marker_layer)
figure

Figure(layout=FigureLayout(height='420px'))